In [1]:
#Import Modules
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
#Set Path for Fast Food Restaurant CSV File
file_path = "Datafiniti_Fast_Food_Restaurants_Jun19.csv"

In [3]:
#Load CSV File to Data Frame
food_data = pd.read_csv(file_path)

food_data

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.396290,-104.696990,Carl's Jr.,80631,CO,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.081350,-108.586890,Del Taco,81505,CO,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.091480,-108.554110,Which Wich,81506,CO,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.763690,-104.776710,Chipotle Mexican Grill,80011,CO,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.740440,-104.926360,Taco Bell,80220,CO,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,AVwd7U0T_7pvs4fz-fS_,2016-05-31T13:33:45Z,2018-12-03T09:04:19Z,1803 Carolina Ave,"American Restaurants,Caterers,Soul Food Restau...",Accommodation & Food Services,Bessemer,US,us/al/bessemer/1803carolinaave/785498836,33.399574,-86.951050,Church's Chicken Of Birmingham,35020,AL,http://www.yellowpages.com/bessemer-al/mip/chu...,http://www.churchs.com
9996,AV1XW1fF3D1zeR_xFKHL,2017-07-18T20:19:50Z,2018-12-02T00:08:36Z,8511 Cantilever Way Ste 105,"Mexican Restaurants,Caterers,Delicatessens,Iri...",Accommodation & Food Services,Raleigh,US,us/nc/raleigh/8511cantileverwayste105/-1596320866,35.884500,-78.737541,Luxury Box,27613-6996,NC,https://www.bbb.org/us/nc/raleigh/profile/spor...,"http://www.theluxuryboxofraleigh.com,http://th..."
9997,AV5UVbkiRxPSIh2RiQxV,2017-09-05T23:05:50Z,2018-12-01T07:38:03Z,2601 Plover Rd,"Carry-out food,Hot dog restaurants,Fast Food R...",Accommodation & Food Services,Plover,US,us/wi/plover/2601ploverrd/-1161002137,44.453180,-89.543170,McDonald's,54467,WI,https://www.yellowpages.com/plover-wi/mip/mcdo...,"http://mcdonalds.com,http://www.mcdonalds.com/..."
9998,AVwc9BxYByjofQCxj_GW,2016-03-31T02:43:12Z,2018-12-01T07:35:22Z,170 E Main St,"Carry-out food,Caterers,Restaurant delivery se...",Accommodation & Food Services,Hendersonville,US,us/tn/hendersonville/170emainst/-66712705,36.308262,-86.607339,Pizza Hut,37075,TN,https://foursquare.com/v/pizza-hut/4c5fffd43a3...,http://www.pizzahut.com


In [4]:
#Drop Rows with Missing Data & Unncecessary Information
fast_food = food_data.dropna()
fast_food = fast_food.drop(["dateAdded", "dateUpdated", "primaryCategories", "country", "keys", "sourceURLs", "websites"], axis = 1)

#Set All Zip Codes to Five Digit Values
zip_codes = []

for code in fast_food["postalCode"]:
    if code.find("-") >= 0:
        code_split = code.split("-")
        
        zip_codes.append(int(code_split[0]))
    else:
        zip_codes.append(int(code))
        
fast_food["postalCode"] = zip_codes
        
fast_food

,id,address,categories,city,latitude,longitude,name,postalCode,province
0,AWrSh_KgsVYjT2BJAzaH,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Greeley,40.396290,-104.696990,Carl's Jr.,80631,CO
1,AWEKlA-LIxWefVJwxG9B,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Grand Junction,39.081350,-108.586890,Del Taco,81505,CO
2,AWrSfAcYsVYjT2BJAzPt,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Grand Junction,39.091480,-108.554110,Which Wich,81506,CO
3,AWrSa3NAQTFama1Xpkbz,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Aurora,39.763690,-104.776710,Chipotle Mexican Grill,80011,CO
4,AWrSaVGzZ4Yw-wtdgcaB,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Denver,39.740440,-104.926360,Taco Bell,80220,CO
...,...,...,...,...,...,...,...,...,...
9995,AVwd7U0T_7pvs4fz-fS_,1803 Carolina Ave,"American Restaurants,Caterers,Soul Food Restau...",Bessemer,33.399574,-86.951050,Church's Chicken Of Birmingham,35020,AL
9996,AV1XW1fF3D1zeR_xFKHL,8511 Cantilever Way Ste 105,"Mexican Restaurants,Caterers,Delicatessens,Iri...",Raleigh,35.884500,-78.737541,Luxury Box,27613,NC
9997,AV5UVbkiRxPSIh2RiQxV,2601 Plover Rd,"Carry-out food,Hot dog restaurants,Fast Food R...",Plover,44.453180,-89.543170,McDonald's,54467,WI
9998,AVwc9BxYByjofQCxj_GW,170 E Main St,"Carry-out food,Caterers,Restaurant delivery se...",Hendersonville,36.308262,-86.607339,Pizza Hut,37075,TN


In [5]:
#Define List of US States
state_list = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA",
              "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK",
              "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

state_list

['AL',
 'AK',
 'AZ',
 'AR',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'HI',
 'ID',
 'IL',
 'IN',
 'IA',
 'KS',
 'KY',
 'LA',
 'ME',
 'MD',
 'MA',
 'MI',
 'MN',
 'MS',
 'MO',
 'MT',
 'NE',
 'NV',
 'NH',
 'NJ',
 'NM',
 'NY',
 'NC',
 'ND',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VT',
 'VA',
 'WA',
 'WV',
 'WI',
 'WY']

In [6]:
#Create State Fast Food Data Frames
for state in state_list:
    if state == "AL":
        AL_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "AK":
        AK_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "AZ":
        AZ_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "AR":
        AR_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "CA":
        CA_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "CO":
        CO_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "CT":
        CT_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "DC":
        DC_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "DE":
        DE_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "FL":
        FL_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "GA":
        GA_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "HI":
        HI_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "ID":
        ID_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "IL":
        IL_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "IN":
        IN_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "IA":
        IA_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "KS":
        KS_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "KY":
        KY_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "LA":
        LA_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "ME":
        ME_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "MD":
        MD_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "MA":
        MA_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "MI":
        MI_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "MN":
        MN_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "MS":
        MS_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "MO":
        MO_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "MT":
        MT_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "NE":
        NE_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "NV":
        NV_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "NH":
        NH_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "NJ":
        NJ_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "NM":
        NM_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "NY":
        NY_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "NC":
        NC_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "ND":
        ND_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "OH":
        OH_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "OK":
        OK_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "OR":
        OR_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "PA":
        PA_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "RI":
        RI_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "SC":
        SC_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "SD":
        SD_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "TN":
        TN_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "TX":
        TX_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "UT":
        UT_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "VT":
        VT_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "VA":
        VA_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "WA":
        WA_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "WV":
        WV_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "WI":
        WI_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
    elif state == "WY":
        WY_food = fast_food[fast_food["province"] == state].reset_index().drop(['index'], axis = 1)
        
print(AL_food)
print(AK_food)
print(AZ_food)
print(AR_food)
print(CA_food)
print(CO_food)
print(CT_food)
print(DC_food)
print(DE_food)
print(FL_food)
print(GA_food)
print(HI_food)
print(ID_food)
print(IL_food)
print(IN_food)
print(IA_food)
print(KS_food)
print(KY_food)
print(LA_food)
print(ME_food)
print(MD_food)
print(MA_food)
print(MI_food)
print(MN_food)
print(MS_food)
print(MO_food)
print(MT_food)
print(NE_food)
print(NV_food)
print(NH_food)
print(NJ_food)
print(NM_food)
print(NY_food)
print(NC_food)
print(ND_food)
print(OH_food)
print(OK_food)
print(OR_food)
print(PA_food)
print(RI_food)
print(SC_food)
print(SD_food)
print(TN_food)
print(TX_food)
print(UT_food)
print(VT_food)
print(VA_food)
print(WA_food)
print(WV_food)
print(WI_food)
print(WY_food)

                       id                  address  \
0    AWkq16muIxWefVJwH5qp       1808 Ashville Road   
1    AVweEDnCByjofQCxu8zM           303 9th Ave SW   
2    AWkqy22X3-Khe5l_21mG            503 1st Ave W   
3    AV1bd6sS3D1zeR_xFZmD             302 W Bypass   
4    AV5TDXKzIxWefVJwnlhH        5356 Highway 67 S   
..                    ...                      ...   
626  AVwdvm3mByjofQCxsF1Z           812 Main St NE   
627  AVwcjrkEkufWRAb5wfkv      279 Highway 78 Loop   
628  AVwdDP38IN2L1WUfrszJ     14471 Us Highway 431   
629  AVwcxaX3kufWRAb5y29P  30500 State Highway 181   
630  AVwd7U0T_7pvs4fz-fS_        1803 Carolina Ave   

                                            categories          city  \
0    Fast Food Restaurants,Chicken Restaurants,Rest...         Leeds   
1    Breakfast Brunch and Lunch Restaurants,Hamburg...     Lafayette   
2    Breakfast Brunch and Lunch Restaurants,Fast Fo...        Reform   
3    Delis And Delicatessens,Restaurant,Carry-out f...     Anda

In [7]:
#Define Variable for Postgres Password
pg_password = ""

In [8]:
#Connect to PostgreSQL Database
pg_engine = create_engine('postgresql://' + pg_password + '@localhost:5432/fast_food_db')

In [9]:
#Create Main Fast Food Table
pg_engine.execute('CREATE TABLE "USA_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL);')

In [10]:
#Add Data to Main Fast Food Table
fast_food.to_sql("USA_Fast_Food", pg_engine, if_exists = 'append', index = False)

In [11]:
#Create State Fast Food Tables
pg_engine.execute('CREATE TABLE "AL_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "AK_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "AZ_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "AR_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "CA_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "CO_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "CT_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "DC_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "DE_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "FL_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "GA_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "HI_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "ID_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "IL_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "IN_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "IA_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "KS_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "KY_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "LA_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "ME_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "MD_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "MA_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "MI_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "MN_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "MS_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "MO_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "MT_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "NE_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "NV_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "NH_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "NJ_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "NM_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "NY_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "NC_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "ND_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "OH_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "OK_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "OR_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "PA_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "RI_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "SC_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "SD_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "TN_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "TX_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "UT_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "VT_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "VA_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "WA_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "WV_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "WI_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')
pg_engine.execute('CREATE TABLE "WY_Fast_Food" ("id" VARCHAR(30) PRIMARY KEY, "address" VARCHAR(200) NOT NULL, "categories" VARCHAR(1000) NOT NULL, "city" VARCHAR(50) NOT NULL, "latitude" FLOAT NOT NULL, "longitude" FLOAT NOT NULL, "name" VARCHAR(100) NOT NULL, "postalCode" INT NOT NULL, "province" VARCHAR(2) NOT NULL, FOREIGN KEY ("id") REFERENCES "USA_Fast_Food" ("id"));')

In [12]:
#Add Data to State Fast Food Tables
AL_food.to_sql("AL_Fast_Food", pg_engine, if_exists = 'append', index = False)
AK_food.to_sql("AK_Fast_Food", pg_engine, if_exists = 'append', index = False)
AZ_food.to_sql("AZ_Fast_Food", pg_engine, if_exists = 'append', index = False)
AR_food.to_sql("AR_Fast_Food", pg_engine, if_exists = 'append', index = False)
CA_food.to_sql("CA_Fast_Food", pg_engine, if_exists = 'append', index = False)
CO_food.to_sql("CO_Fast_Food", pg_engine, if_exists = 'append', index = False)
CT_food.to_sql("CT_Fast_Food", pg_engine, if_exists = 'append', index = False)
DC_food.to_sql("DC_Fast_Food", pg_engine, if_exists = 'append', index = False)
DE_food.to_sql("DE_Fast_Food", pg_engine, if_exists = 'append', index = False)
FL_food.to_sql("FL_Fast_Food", pg_engine, if_exists = 'append', index = False)
GA_food.to_sql("GA_Fast_Food", pg_engine, if_exists = 'append', index = False)
HI_food.to_sql("HI_Fast_Food", pg_engine, if_exists = 'append', index = False)
ID_food.to_sql("ID_Fast_Food", pg_engine, if_exists = 'append', index = False)
IL_food.to_sql("IL_Fast_Food", pg_engine, if_exists = 'append', index = False)
IN_food.to_sql("IN_Fast_Food", pg_engine, if_exists = 'append', index = False)
IA_food.to_sql("IA_Fast_Food", pg_engine, if_exists = 'append', index = False)
KS_food.to_sql("KS_Fast_Food", pg_engine, if_exists = 'append', index = False)
KY_food.to_sql("KY_Fast_Food", pg_engine, if_exists = 'append', index = False)
LA_food.to_sql("LA_Fast_Food", pg_engine, if_exists = 'append', index = False)
ME_food.to_sql("ME_Fast_Food", pg_engine, if_exists = 'append', index = False)
MD_food.to_sql("MD_Fast_Food", pg_engine, if_exists = 'append', index = False)
MA_food.to_sql("MA_Fast_Food", pg_engine, if_exists = 'append', index = False)
MI_food.to_sql("MI_Fast_Food", pg_engine, if_exists = 'append', index = False)
MN_food.to_sql("MN_Fast_Food", pg_engine, if_exists = 'append', index = False)
MS_food.to_sql("MS_Fast_Food", pg_engine, if_exists = 'append', index = False)
MO_food.to_sql("MO_Fast_Food", pg_engine, if_exists = 'append', index = False)
MT_food.to_sql("MT_Fast_Food", pg_engine, if_exists = 'append', index = False)
NE_food.to_sql("NE_Fast_Food", pg_engine, if_exists = 'append', index = False)
NV_food.to_sql("NV_Fast_Food", pg_engine, if_exists = 'append', index = False)
NH_food.to_sql("NH_Fast_Food", pg_engine, if_exists = 'append', index = False)
NJ_food.to_sql("NJ_Fast_Food", pg_engine, if_exists = 'append', index = False)
NM_food.to_sql("NM_Fast_Food", pg_engine, if_exists = 'append', index = False)
NY_food.to_sql("NY_Fast_Food", pg_engine, if_exists = 'append', index = False)
NC_food.to_sql("NC_Fast_Food", pg_engine, if_exists = 'append', index = False)
ND_food.to_sql("ND_Fast_Food", pg_engine, if_exists = 'append', index = False)
OH_food.to_sql("OH_Fast_Food", pg_engine, if_exists = 'append', index = False)
OK_food.to_sql("OK_Fast_Food", pg_engine, if_exists = 'append', index = False)
OR_food.to_sql("OR_Fast_Food", pg_engine, if_exists = 'append', index = False)
PA_food.to_sql("PA_Fast_Food", pg_engine, if_exists = 'append', index = False)
RI_food.to_sql("RI_Fast_Food", pg_engine, if_exists = 'append', index = False)
SC_food.to_sql("SC_Fast_Food", pg_engine, if_exists = 'append', index = False)
SD_food.to_sql("SD_Fast_Food", pg_engine, if_exists = 'append', index = False)
TN_food.to_sql("TN_Fast_Food", pg_engine, if_exists = 'append', index = False)
TX_food.to_sql("TX_Fast_Food", pg_engine, if_exists = 'append', index = False)
UT_food.to_sql("UT_Fast_Food", pg_engine, if_exists = 'append', index = False)
VT_food.to_sql("VT_Fast_Food", pg_engine, if_exists = 'append', index = False)
VA_food.to_sql("VA_Fast_Food", pg_engine, if_exists = 'append', index = False)
WA_food.to_sql("WA_Fast_Food", pg_engine, if_exists = 'append', index = False)
WV_food.to_sql("WV_Fast_Food", pg_engine, if_exists = 'append', index = False)
WI_food.to_sql("WI_Fast_Food", pg_engine, if_exists = 'append', index = False)
WY_food.to_sql("WY_Fast_Food", pg_engine, if_exists = 'append', index = False)